In [ ]:
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y
!pip install selenium webdriver-manager

--2025-03-22 14:31:49--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 172.253.122.93, 172.253.122.190, 172.253.122.91, ...
Connecting to dl.google.com (dl.google.com)|172.253.122.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 114483440 (109M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb.1’

google-chrome-stabl 100%[===================>] 109.18M   363MB/s    in 0.3s    

2025-03-22 14:31:50 (363 MB/s) - ‘google-chrome-stable_current_amd64.deb.1’ saved [114483440/114483440]

(Reading database ... 126364 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (134.0.6998.165-1) over (134.0.6998.165-1) ...
Setting up google-chrome-stable (134.0.6998.165-1) ...
Processing triggers for mailcap (3.70+nmu1ubuntu1) ...
Processing triggers for man-db (2.10.2-1) ...
Reading packa

In [ ]:
!pip install selenium beautifulsoup4 requests

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

In [ ]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

base_url = "https://svichado.com/all-books?limit=100&page={}"  # Adjust URL structure as needed

all_links = []
page_num = 1

while True:
    # Load the current page
    url = base_url.format(page_num)
    driver.get(url)
    time.sleep(3)  # Wait for content to load

    # Parse the page with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Extract book links from the current page
    book_links = [a["href"] for a in soup.select("div.product-thumb h4.name a")]

    if not book_links:
        break

    all_links.extend(book_links)
    print(f"Page {page_num}: {len(book_links)} books found")

    page_num += 1

driver.quit()

Page 1: 100 books found
Page 2: 100 books found
Page 3: 100 books found
Page 4: 100 books found
Page 5: 100 books found
Page 6: 100 books found
Page 7: 100 books found
Page 8: 100 books found
Page 9: 100 books found
Page 10: 100 books found
Page 11: 100 books found
Page 12: 100 books found
Page 13: 100 books found
Page 14: 100 books found
Page 15: 100 books found
Page 16: 21 books found

All Book Links:
https://svichado.com/hydojnya_lsteratyra/prytchi_ta_opovidannya/100-malih-istorij-pro-veliki-pravdi.html
https://svichado.com/vydannya_dlya_molodi_i_ditey/molodschyj_schkilnyj_vik/100-ulyublenih-bblynih-story.html
https://svichado.com/simya_ta_vyhovannya/podryjjya_simya/12-nepravd-jaki-choloviki-govorjat-druzhinam.html
https://svichado.com/svyati_i_vyznachni_postati/postati/150-dumok-mitropolita-andreja-sheptic-kogo.html
https://svichado.com/vydannya_dlya_molodi/16-krokiv-do-zrilosti.html
https://svichado.com/hydojnya_lsteratyra/prytchi_ta_opovidannya/365-korotkih-istorij-dlja-dushi-bru

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

chrome_options = Options()
chrome_options.add_argument("--headless")  # No GUI
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--window-size=1920,1080")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# === Scraping book details ===
all_books_info = []

for book_url in all_links:
    driver.get(book_url)
    time.sleep(1.5)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Breadcrumbs for category info
    breadcrumbs = [crumb.text.strip() for crumb in soup.select("ul.breadcrumb li a")]
    category = breadcrumbs[1] if len(breadcrumbs) > 1 else None
    subcategory = breadcrumbs[2] if len(breadcrumbs) > 2 else None
    sub_subcategory = breadcrumbs[3] if len(breadcrumbs) > 3 else None
    sub_sub_subcategory = breadcrumbs[4] if len(breadcrumbs) > 4 else None

    def extract_detail(label):
        try:
            row = soup.select_one(f"li:has(span:contains('{label}'))")
            return row.select_one("span.red").text.strip() if row else None
        except:
            return None

    def extract_text(selector):
        element = soup.select_one(selector)
        return element.text.strip() if element else None

    # Main book data
    title = extract_text("h1.product-name")
    details_list = soup.select("li")

    def get_value(label):
        for item in details_list:
            if label in item.text:
                red_span = item.select_one("span.red")
                return red_span.text.strip() if red_span else None
        return None

    book_info = {
        "Title": title,
        "Author": get_value("Автор"),
        "Category": category,
        "Subcategory": subcategory,
        "Sub-Subcategory": sub_subcategory,
        "Sub-Sub-Subcategory": sub_sub_subcategory,
        "Pages": get_value("Кількість сторінок"),
        "Cover Type": get_value("Палітурка"),
        "Year": get_value("Рік видання"),
        "ISBN": get_value("ISBN"),
        "Translator": get_value("Переклад"),
        "Format": get_value("Формат"),
        "Availability": get_value("Наявність"),
        "Price": extract_text("h2.price"),
        "Description": soup.select_one("#tab-description").text.strip() if soup.select_one("#tab-description") else None
    }

    all_books_info.append(book_info)

driver.quit()


✅ Scraped: 100 малих історій про великі правди
✅ Scraped: 100 улюблених біблійних історій
✅ Scraped: 12 неправд, які чоловіки говорять дружинам
✅ Scraped: 150 думок митрополита Андрея Шептицького
✅ Scraped: 16 кроків до зрілості
✅ Scraped: 365 коротких історій для душі
✅ Scraped: 365 притч на щодень
✅ Scraped: 7 Різдвяних історій
✅ Scraped: Акафісник. Книга 1
✅ Scraped: Акафісник. Книга 2
✅ Scraped: Акафіст до Христових страстей
✅ Scraped: Акафіст за померлого
✅ Scraped: Ангел Хоронитель. Молитовник для дітей 5-9 років (Електронна книга: PDF)
✅ Scraped: Ангел Хоронитель. Молитовник для дітей 5-9 років.
✅ Scraped: Ангели - вісники світла
✅ Scraped: Ангели — помічники Бога і друзі людини
✅ Scraped: Андрей Шептицький. Визначні постаті
✅ Scraped: Антикризовий біблійний порадник
✅ Scraped: Апостол
✅ Scraped: Байки харківські (Електронна книга: PDF)
✅ Scraped: Батьківство не для боягузів
✅ Scraped: Бачити в темряві : Досвід незрячої людини.
✅ Scraped: Бен Гур
✅ Scraped: Бесіди з етики
✅ Scra

In [ ]:
import pandas as pd
df = pd.DataFrame(all_books_info)
df.to_csv('book_info.csv')

In [ ]:
import re

# Load and normilize dataset
df = pd.read_csv("/content/drive/MyDrive/Diploma/book_info.csv")
df = df[df["Title"].fillna("").str.strip() != ""]
df.reset_index(drop=True, inplace=True)
df["Text"] = (df["Title"].fillna("") + " " + df["Description"].fillna("")).str.lower()

# Defining New Columns
def get_target_audience(text):
    if any(w in text for w in ["дитина", "казка", "малюк", "абетка", "школа", "дитячий", "вірш"]):
        return "Діти"
    if any(w in text for w in ["молодь", "підліток", "зрілість", "дорослішання"]):
        return "Підлітки"
    if any(w in text for w in ["подружжя", "шлюб", "виховання", "родина", "батьки", "мати"]):
        return "Сім'ї"
    if any(w in text for w in ["душа", "молитва", "покаяння", "таїнство", "піст", "богослов", "служіння"]):
        return "Дорослі"
    return "Загальна аудиторія"

def get_tone(text):
    if any(w in text for w in ["надихаючі", "впевненість", "надія", "серце", "щастя"]):
        return "Натхненний"
    if any(w in text for w in ["оповідання", "історія", "притча", "повість", "сюжет"]):
        return "Оповідний"
    if any(w in text for w in ["поради", "практичний", "як", "чому", "інструкція"]):
        return "Практичний"
    if any(w in text for w in ["теологія", "богослов", "доктрина", "вчення", "катехизм"]):
        return "Доктринальний"
    if any(w in text for w in ["поезія", "вірш", "метафора", "поетично"]):
        return "Поетичний"
    return "Нейтральний"

def classify_biblical_level(text):
    high_roots = [
        "біблі", "євангел", "святе письмо", "псалм", "псалтир", "пророк", "апостол",
        "тлумачен", "коментар", "старий завіт", "новий завіт", "богослов", "юдей", "слово боже"
    ]
    medium_roots = [
        "притч", "роздум", "заповід", "святий", "катехизм", "духовн", "повчан", "доктрин", "догма"
    ]
    low_roots = [
        "мораль", "мудр", "цінн", "щаст", "краса", "любов", "натхнен", "емоці", "розвиток", "поради"
    ]
    extra_high = ["господь", "ісус", "христос", "спаситель", "учні", "воскресіння", "сповідь"]
    extra_medium = ["чесноти", "світогляд", "християнськ", "віровчення"]

    high_score = sum(1 for root in high_roots if re.search(root, text))
    medium_score = sum(1 for root in medium_roots if re.search(root, text))
    low_score = sum(1 for root in low_roots if re.search(root, text))

    high_score += sum(1 for kw in extra_high if kw in text)
    medium_score += sum(1 for kw in extra_medium if kw in text)

    if high_score >= max(medium_score, low_score) and high_score > 0:
        return "Високий"
    elif medium_score >= max(high_score, low_score) and medium_score > 0:
        return "Середній"
    elif low_score > 0:
        return "Низький"
    else:
        return "Дуже низький"

def get_application_type(text):
    if "день" in text and "роздуми" in text:
        return "Щоденний молитовник"
    if any(w in text for w in ["вивчення", "уроки", "курс", "тлумачення"]):
        return "Навчання/Вивчення"
    if any(w in text for w in ["біографія", "житія", "історія життя"]):
        return "Біографія"
    if any(w in text for w in ["молитви", "молитовник", "прохання", "молебень", "акафісник"]):
        return "Молитовник"
    if any(w in text for w in ["казка", "повість", "розповідь"]):
        return "Оповідання"
    return "Загальне"

def get_event_relevance(text):
    if any(w in text for w in ["різдв", "новий рік", "коляда"]):
        return "Різдво"
    if "великдень" in text:
        return "Великдень"
    if any(w in text for w in ["піст", "страсті", "страсна"]):
        return "Піст"
    if any(w in text for w in ["випуск", "школа", "навчання"]):
        return "Шкільний період"
    return "Без події"

def get_content_format(text):
    if any(w in text for w in ["поезія", "вірш", "вірші"]):
        return "Поезія"
    if any(w in text for w in ["оповідання", "повість", "розповідь", "притча"]):
        return "Оповідання"
    if any(w in text for w in ["молитва", "акафісник", "піснеспів"]):
        return "Молитва"
    if any(w in text for w in ["лист", "діалог"]):
        return "Діалог/Лист"
    return "Проза"

df["Цільова аудиторія"] = df["Text"].apply(get_target_audience)
df["Тон"] = df["Text"].apply(get_tone)
df["Біблійний рівень"] = df["Text"].apply(classify_biblical_level)
df["Тип застосування"] = df["Text"].apply(get_application_type)
df["Подієва прив'язка"] = df["Text"].apply(get_event_relevance)
df["Формат контенту"] = df["Text"].apply(get_content_format)

df.drop(columns=["Text"], inplace=True)
df.to_csv("book_info_enriched_ukr_final.csv", index=False)

✅ Готово! Збережено як 'book_info_enriched_ukr_final.csv'


In [1]:
from functools import reduce
from tqdm import tqdm

# Load and normilize dataset
book_info = pd.read_csv("book_info_enriched_ukr_final.csv")
book_info["Title_clean"] = book_info["Title"].str.strip().str.lower()

# Load and normilize sales dataset
sales_dfs = []
sales_book_col = "Назва"  # Book title column in sales files

for year in range(2020, 2025):
    file_path = f"/content/drive/MyDrive/Diploma/Books_{year}.xlsx"
    df_year = pd.read_excel(file_path, skiprows=6)

    df_year.rename(columns={
        col: f"{col}_{year}" for col in df_year.columns if col != sales_book_col
    }, inplace=True)

    sales_dfs.append(df_year)

all_sales = reduce(lambda left, right: pd.merge(left, right, on=sales_book_col, how="outer"), sales_dfs)
all_sales["Book_clean"] = all_sales[sales_book_col].str.strip().str.lower()

FileNotFoundError: [Errno 2] No such file or directory: 'book_info_enriched_ukr_final.csv'

In [ ]:
# Fuzzy search
all_sales["matched_title"] = None
all_sales["match_score"] = 0.0

# Match each sales title to the best fitting book_info
for i, sale_row in tqdm(all_sales.iterrows(), total=len(all_sales), desc="Matching Titles"):
    sale_title = str(sale_row["Book_clean"]).strip().lower()

    best_match = None
    best_score = 0

    for info_title_raw in book_info["Title_clean"]:
        info_title = str(info_title_raw).strip().lower()

        if info_title and sale_title and (info_title in sale_title or sale_title in info_title):
            score = len(info_title) / len(sale_title) if len(sale_title) > 0 else 0
            if score > best_score:
                best_score = score
                best_match = info_title

    if best_match:
        all_sales.at[i, "matched_title"] = best_match
        all_sales.at[i, "match_score"] = best_score

matched_sales = all_sales[all_sales["match_score"] > 0]

# Merge only matched rows with book_info
merged = pd.merge(matched_sales, book_info, left_on="matched_title", right_on="Title_clean", how="inner")


merged.drop(columns=["Book_clean", "Title_clean", "matched_title", "match_score"], inplace=True)
book_info_cols = list(book_info.columns)
book_info_cols = [col for col in book_info.columns if col in merged.columns]
sales_cols = [col for col in merged.columns if col not in book_info_cols]
merged = merged[book_info_cols + sales_cols]


merged.to_csv("book_sales_merged_fuzzy_titles.csv", index=False)

Matching Titles: 100%|██████████| 1555/1555 [00:04<00:00, 371.76it/s]

✅ DONE! Merged using fuzzy title matching and saved to 'book_sales_merged_fuzzy_titles.csv'


In [ ]:
df = merged
years = ["2020", "2021", "2022", "2023", "2024"]
sales_cols = [col for col in df.columns if any(year in col for year in years)]

for col in sales_cols:
    df[col] = df[col].apply(lambda x: x if pd.isna(x) else (x if x >= 0 else 0))

df["Total_Sales"] = df[sales_cols].sum(axis=1)
df.to_csv("book_sales_merged_final_no_negatives.csv", index=False)

✅ Total_Sales column added.


In [ ]:
import numpy as np

# Load dataset
df = pd.read_csv("book_sales_merged_final_no_negatives.csv")

years = ["2020", "2021", "2022", "2023", "2024"]
sales_cols = [col for col in df.columns if any(year in col for year in years)]

for col in sales_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col] = df[col].replace("", np.nan)

df_cleaned = df.dropna(subset=sales_cols, how="all")
df_cleaned.to_csv("book_sales_merged_final_with_sales.csv", index=False)
print(df_cleaned)


     Unnamed: 0                                              Title  \
0          1231                           10 способів змінити себе   
1          1232                         10 способів стати щасливим   
2          1233                                100 кроків до щастя   
3          1234                   100 кроків звільнення від страху   
4             0                100 малих історій про великі правди   
..          ...                                                ...   
926        1060  Якби ж ми це знали, перш ніж наші діти стали п...   
927        1062  Якби ж ми це знали, перш ніж стати батьками (Е...   
928        1228                Якщо ви надмірно переживаєте за все   
929        1230              Якщо ви пережили психотравмуючу подію   
930        1063   Якщо справді мене кохаєш. Сто запитань про любов   

                         Author                  Category  \
0           Войцех Єнджеєвський                       NaN   
1              Яцек Кшиштофович      

In [ ]:
# Load dataset
df = pd.read_csv("book_sales_merged_final_with_sales.csv")

df = df.drop(columns=["Unnamed: 0"])
df.reset_index(drop=True, inplace=True)
df=df.dropna(axis=1, how='all')

df.to_csv("book_sales_merged_final_with_sales_clean.csv", index=False)

                                                 Title  \
0                             10 способів змінити себе   
1                           10 способів стати щасливим   
2                                  100 кроків до щастя   
3                     100 кроків звільнення від страху   
4                  100 малих історій про великі правди   
..                                                 ...   
926  Якби ж ми це знали, перш ніж наші діти стали п...   
927  Якби ж ми це знали, перш ніж стати батьками (Е...   
928                Якщо ви надмірно переживаєте за все   
929              Якщо ви пережили психотравмуючу подію   
930   Якщо справді мене кохаєш. Сто запитань про любов   

                         Author                  Category  \
0           Войцех Єнджеєвський                       NaN   
1              Яцек Кшиштофович                       NaN   
2                    Ів Бульвен                Духовність   
3                    Ів Бульвен                       NaN  

In [ ]:
author_counts = df["Author"].value_counts()
df["Author Score"] = df["Author"].map(author_counts)
df["Author Score"] = df["Author Score"].fillna(0).astype(int)

cols = list(df.columns)
author_idx = cols.index("Author")
cols.insert(author_idx + 1, cols.pop(cols.index("Author Score")))

df = df[cols]

print(df)
df.to_csv("book_sales_final with author score.csv", index=False)

                                                 Title  \
0                             10 способів змінити себе   
1                           10 способів стати щасливим   
2                                  100 кроків до щастя   
3                     100 кроків звільнення від страху   
4                  100 малих історій про великі правди   
..                                                 ...   
926  Якби ж ми це знали, перш ніж наші діти стали п...   
927  Якби ж ми це знали, перш ніж стати батьками (Е...   
928                Якщо ви надмірно переживаєте за все   
929              Якщо ви пережили психотравмуючу подію   
930   Якщо справді мене кохаєш. Сто запитань про любов   

                         Author  Author Score                  Category  \
0           Войцех Єнджеєвський             2                       NaN   
1              Яцек Кшиштофович             1                       NaN   
2                    Ів Бульвен             2                Духовність   
3  